In [75]:
import os

In [76]:
%pwd


'C:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [77]:
os.chdir('../.')

In [78]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [79]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [80]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    val_root_dir: Path
    STATUS_FILE: str
    customer_data: Path
    all_schema: dict
    customer_data: Path
    





In [81]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [82]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([self.config.artifacts_root])

        data_validation_config = DataValidationConfig(
            val_root_dir=Path(config.val_root_dir),
            STATUS_FILE=Path(config.STATUS_FILE),
            customer_data=Path(config.customer_data),
            all_schema=schema,
        )

        return data_validation_config
        
        

In [83]:
import os
import pandas as pd
from pathlib import Path
from BankProducts import  logger

In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        logger.info(f"Data Validation started ")
        
    def validate_all_columns(self) -> bool:
        try:
            data = pd.read_csv(self.config.customer_data)
            logger.info(f"Data loaded from {self.config.customer_data}")
            
            # convert the Transaction Date column to datetime format
            data['transactiondate'] = pd.to_datetime(data['transactiondate'])
            # Check if the data is empty
            logger.info(f"Checking if data is empty")                       
            if data.empty:          
                return False
            logger.info(f"Data loaded successfully with {data.shape[0]} rows and {data.shape[1]} columns")      
            # Check if all required columns are present                     
            
            
            if data.empty:
                raise ValueError("Customer data is empty.")
            all_cols = set(data.columns)
            all_schema = set(self.config.all_schema.keys())
            logger.info(f"Columns in data: {all_cols}")

            # Check for missing columns in data
            missing_in_data = all_schema - all_cols
            
            # Check for extra columns in data
            extra_in_data = all_cols - all_schema
            logger.info(f"checking for missing columns in data: {missing_in_data}")

            validation_status = True
            if missing_in_data or extra_in_data:
                validation_status = False
                logger.error(f"Missing columns in data: {missing_in_data}")

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}\n")
                if missing_in_data:
                    f.write(f"Missing columns in data: {missing_in_data}\n")
                if extra_in_data:
                    f.write(f"Extra columns in data: {extra_in_data}\n")

            logger.info(f"All columns validation status: {validation_status}")
            return validation_status

        except Exception as e:
            raise e
    
    
    def validate_all_column(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.customer_data)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e
 

In [85]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    
    # Validate CSV files
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_all_column()
    
except Exception as e:
    print(f"Validation failed: {e}")
# The code above is a complete example of how to validate the existence and content of CSV files and database tables.

[2025-06-02 23:27:19,962: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-02 23:27:19,972: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-02 23:27:19,979: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-02 23:27:19,982: INFO: common: created directory at: artifacts]
[2025-06-02 23:27:19,985: INFO: common: created directory at: artifacts]
[2025-06-02 23:27:19,987: INFO: 1370065407: Data Validation started ]
[2025-06-02 23:27:20,151: INFO: 1370065407: Data loaded from artifacts\data_ingestion\customer_data.csv]
Validation failed: 'Transactiondate'


In [86]:
df = pd.read_csv(data_validation_config.customer_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transactionid       20000 non-null  int64  
 1   customerid          20000 non-null  int64  
 2   transactiondate     20000 non-null  object 
 3   transactiontype     20000 non-null  object 
 4   amount              20000 non-null  float64
 5   productcategory     20000 non-null  object 
 6   productsubcategory  20000 non-null  object 
 7   branchcity          20000 non-null  object 
 8   branchlat           20000 non-null  float64
 9   branchlong          20000 non-null  float64
 10  channel             20000 non-null  object 
 11  currency            20000 non-null  object 
 12  creditcardfees      20000 non-null  float64
 13  insurancefees       20000 non-null  float64
 14  latepaymentamount   20000 non-null  float64
 15  customerscore       20000 non-null  int64  
 16  mont